In [6]:
import torch
from torch import nn

print(torch.__version__)

1.13.1+cpu


## SRGAN Architecture

<img src="SRGAN-Architecture.png">

### 1. Defining classes for the architechture

In [10]:
class ConvBlock(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        discriminator=False,
        use_act=True,
        use_bn=True,
        **kwargs,
    ):
        # Initialize the convolutional block
        super().__init__()
        self.use_act = use_act
        # Define the convolutional layer
        self.cnn = nn.Conv2d(in_channels, out_channels, **kwargs, bias=not use_bn)
        # Define the batch normalization layer if used
        self.bn = nn.BatchNorm2d(out_channels) if use_bn else nn.Identity()
        # Define the activation function based on discriminator or generator
        self.act = (
            nn.LeakyReLU(0.2, inplace=True)
            if discriminator
            else nn.PReLU(num_parameters=out_channels)
        )

    def forward(self, x):
        # Forward pass through the convolutional block
        return self.act(self.bn(self.cnn(x))) if self.use_act else self.bn(self.cnn(x))


class UpsampleBlock(nn.Module):
    def __init__(self, in_c, scale_factor):
        # Initialize the upsampling block
        super().__init__()
        # Define the convolutional layer for upsampling
        self.conv = nn.Conv2d(in_c, in_c * scale_factor ** 2, 3, 1, 1)
        # Define the pixel shuffle layer for upsampling
        self.ps = nn.PixelShuffle(scale_factor)  # in_c * 4, H, W --> in_c, H*2, W*2
        # Define the activation function
        self.act = nn.PReLU(num_parameters=in_c)

    def forward(self, x):
        # Forward pass through the upsampling block
        return self.act(self.ps(self.conv(x)))


class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        # Initialize the residual block
        super().__init__()
        # Define the first convolutional block
        self.block1 = ConvBlock(
            in_channels,
            in_channels,
            kernel_size=3,
            stride=1,
            padding=1
        )
        # Define the second convolutional block
        self.block2 = ConvBlock(
            in_channels,
            in_channels,
            kernel_size=3,
            stride=1,
            padding=1,
            use_act=False,
        )

    def forward(self, x):
        # Forward pass through the residual block
        out = self.block1(x)
        out = self.block2(out)
        return out + x


class Generator(nn.Module):
    def __init__(self, in_channels=3, num_channels=64, num_blocks=16):
        # Initialize the generator
        super().__init__()
        # Define the initial convolutional block
        self.initial = ConvBlock(in_channels, num_channels, kernel_size=9, stride=1, padding=4, use_bn=False)
        # Define the residual blocks
        self.residuals = nn.Sequential(*[ResidualBlock(num_channels) for _ in range(num_blocks)])
        # Define the convolutional block after residuals
        self.convblock = ConvBlock(num_channels, num_channels, kernel_size=3, stride=1, padding=1, use_act=False)
        # Define the upsampling blocks
        self.upsamples = nn.Sequential(UpsampleBlock(num_channels, 2), UpsampleBlock(num_channels, 2))
        # Define the final convolutional layer
        self.final = nn.Conv2d(num_channels, in_channels, kernel_size=9, stride=1, padding=4)

    def forward(self, x):
        # Forward pass through the generator
        initial = self.initial(x)
        x = self.residuals(initial)
        x = self.convblock(x) + initial
        x = self.upsamples(x)
        return torch.tanh(self.final(x))


class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 64, 128, 128, 256, 256, 512, 512]):
        # Initialize the discriminator
        super().__init__()
        blocks = []
        for idx, feature in enumerate(features):
            # Define the convolutional blocks for the discriminator
            blocks.append(
                ConvBlock(
                    in_channels,
                    feature,
                    kernel_size=3,
                    stride=1 + idx % 2,
                    padding=1,
                    discriminator=True,
                    use_act=True,
                    use_bn=False if idx == 0 else True,
                )
            )
            in_channels = feature

        self.blocks = nn.Sequential(*blocks)
        # Define the classifier for the discriminator
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((6, 6)),
            nn.Flatten(),
            nn.Linear(512*6*6, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 1),
        )

    def forward(self, x):
        # Forward pass through the discriminator
        x = self.blocks(x)
        return self.classifier(x)

def test():
    # Test the generator and discriminator
    low_resolution = 24  # 96x96 -> 24x24
    with torch.cuda.amp.autocast():
        x = torch.randn((5, 3, low_resolution, low_resolution))
        gen = Generator()
        gen_out = gen(x)
        disc = Discriminator()
        disc_out = disc(gen_out)

        print(gen_out.shape)
        print(disc_out.shape)


test()

torch.Size([5, 3, 96, 96])
torch.Size([5, 1])
